In [11]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima

In [3]:
data = pd.read_csv('model_data3.csv')

In [15]:
# Creating more condensed table and dropping null values

new = data[['Supplier', 'days_late', 'Invoice ($)', 'Invoice Qty', 'Purchase Order $', 'Purchase Order Qty']].dropna()
new

,Supplier,days_late,Invoice ($),Invoice Qty,Purchase Order $,Purchase Order Qty
1,ELMO SWEDEN AB,-2.0,295.96,51.99,212.50,50.00
2,SHORELINE CONTAINER LLC,0.0,672.25,125.00,672.25,125.00
3,ROYAL TECHNOLOGIES CORPORATION,-1.0,852.00,400.00,852.00,400.00
5,JEMNI INC,0.0,155.84,600.00,155.84,600.00
6,ROYAL TECHNOLOGIES CORPORATION,-1.0,325.00,100.00,325.00,100.00
...,...,...,...,...,...,...
67198,MEDWAY PLASTICS CORP,-306.0,11197.44,1152.00,11957.76,1152.00
67228,GRAND RAPIDS FOAM TECHNOLOGIES,-357.0,1676.40,120.00,1676.40,120.00
67255,MAYER-PAETZ INC,-298.0,57.58,3.50,57.58,3.50
67265,APPLIED INDUSTRIAL TECH,-189.0,31920.00,8000.00,31920.00,8000.00


## Correlation Between Price/Quantity and On Time

In [53]:
# Correlation of full dataset

def full(df, column):
    return np.corrcoef(df['days_late'] > 0, new[column])

array([[1.        , 0.14840799],
       [0.14840799, 1.        ]])

In [76]:
# Correlation of price with different percentiles late or not


def percentile(df, column):
    greater_25 = df.loc[df[column] > df[column].quantile(q = .25)]
    greater_75 = df.loc[df[column] > df[column].quantile(q = .75)]
    middle_25_75 = df.loc[(new[column] > df[column].quantile(q = .25)) & ((df[column] < df[column].quantile(q = .75)))]
    less_75 = df.loc[new[column] < df[column].quantile(q = .75)]
    less_25 = df.loc[new[column] < df[column].quantile(q = .25)]

    count = 0
    for frame in [greater_25, greater_75, middle_25_75, less_25, less_75]:
        count += 1
        print(f'{count}: {np.corrcoef(frame["days_late"] > 0, frame[column])[0][1]}')


print(percentile(new, 'Invoice Qty'))



1: 0.01717745566491543
2: 0.03432127888712984
3: 0.005819377962920769
4: -0.19103142615968757
5: -0.0552780840775975
None


In [69]:
less_25.groupby('Supplier').agg('count').sort_values(by = 'days_late', ascending = False)

,days_late,Invoice ($),Invoice Qty,Purchase Order $,Purchase Order Qty
Supplier,,,,,
ROYAL TECHNOLOGIES CORPORATION,7466,7466,7466,7466,7466
SHORELINE CONTAINER LLC,2807,2807,2807,2807,2807
GRAND RAPIDS FOAM TECHNOLOGIES,889,889,889,889,889
WRIGHT COATING COMPANY,582,582,582,582,582
ACTION INDUSTRIAL SUPPLY CO,408,408,408,408,408
MANSCO A DIVISION OF FASTENAL,315,315,315,315,315
ELMO SWEDEN AB,281,281,281,281,281
ERVINS GROUP LLC,218,218,218,218,218
MEDWAY PLASTICS CORP,199,199,199,199,199


In [103]:
# Average PPU vs On Time

data['Late'] = [1 if line > 0 else 0 for line in data['days_late']]
a = data[['Supplier', 'avg_unit_price', 'Late']].groupby('Supplier').agg('mean').dropna()
np.corrcoef(a['avg_unit_price'], a['Late'])

## Regression

In [115]:
import statsmodels.api as sm

X = data[['Supplier', 'Material', 'Material_Group', 'days_late', 'dayofweek', 'error_code', 'avg_unit_price', 'delivery_month', 'delivery_season', 'Invoice Qty', 'Invoice ($)']]
Y = data['Late']

est = sm.OLS(Y, X)
est2 = est.fit()
print(est2.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [180]:
from pmdarima.arima import auto_arima

Royal = data[data['Supplier'] == "ROYAL TECHNOLOGIES CORPORATION"]
Royal = Royal[['Late', 'Posting_Date']].reset_index()
Royal['Date'] = pd.to_datetime(Royal['Posting_Date'])
# Royal['Date'] = pd.DatetimeIndex(Royal['Posting_Date']).to_period('M')

df3 = Royal.groupby([pd.Grouper(key='Date', freq='MS')])['Late'].agg(['mean']).reset_index()

time_series = df3[['mean', 'Date']]

auto_arima(time_series)

# time = pd.to_datetime(Royal['Posting_Date'])


TypeError: The DType <class 'numpy.dtype[float64]'> could not be promoted by <class 'numpy.dtype[datetime64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[float64]'>, <class 'numpy.dtype[datetime64]'>)

PeriodIndex(['2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07',
             '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
             '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
             '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
             '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07',
             '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01',
             '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07',
             '2022-08'],
            dtype='period[M]', name='Date')